In [1]:
!pip install transformers

In [2]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
import time

In [3]:
# Install TensorFlow
!pip install tensorflow

# Import library yang diperlukan
import tensorflow as tf
from google.colab import drive

In [4]:

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the bert tokenizer
from transformers import AutoTokenizer,AdamW,BertForQuestionAnswering, AutoModel
tokenizer = AutoTokenizer.from_pretrained("Rifky/Indobert-QA")

# Load the fine-tuned modeol
model = torch.load("/content/drive/MyDrive/model3_1.9",map_location=torch.device('cpu'))
#model = AutoModel.from_pretrained("Rifky/Indobert-QA")
model.eval()

In [6]:
#fungsi prediksi

def predict(context,query):

  inputs = tokenizer.encode_plus(query, context, return_tensors='pt')

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
  answer_end = torch.argmax(outputs[1]) + 1

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return 2 * (prec * rec) / (prec + rec)

def compute_precision(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return prec

def compute_recall(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)

  common_tokens = set(pred_tokens) & set(truth_tokens)

  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)

  return rec

In [7]:
def give_an_answer(context,query,answer):
  prediction = predict(context,query)
  em_score = compute_exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)
  prec = compute_precision(prediction, answer)
  rec = compute_recall(prediction, answer)

  print(f"Question: {query}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print(f"EM: {em_score}")
  print(f"F1: {f1_score}")
  print(f"Precision: {prec}")
  print(f"Recall: {rec}")
  print("\n")

QUESTION ANSWERING

In [27]:
context = "Hama dan penyakit tumbuhan merupakan jenis organisme pengganggu tumbuhan (OPT), selain gulma. Serangan hama dan penyakit pada tanaman dapat menyebabkan kerugian besar pada tanaman dan dapat mengancam perekonomian petani. Penyebaran hama dan penyakit tanaman meningkat drastis dalam beberapa tahun terakhir. Hama dan penyakit tanaman mudah menyebar ke beberapa negara dan mencapai proporsi epidemi. Belalang, lalat buah, ulat grayak, penyakit antaknose, fuso, penyakit virus kerdil, busuk buah adalah beberapa hama dan penyakit tanaman yang paling merusak. Tiga cara penyebaran hama dan penyakit tanaman yaitu dengan cara: 1) perdagangan atau migrasi 2) pengaruh lingkungan, seperti faktor cuaca, angin, percikan air hujan, dan 3) faktor biotik berupa: serangga atau vektor lainnya."
queries = ["Apa yang dimaksud dengan Hama dan Penyakit Tanaman?",
           "Apa saja beberapa hama dan penyakit tanaman yang merusak?",
           "Bagaimana cara penyebaran hama dan penyakit tanaman?"
           ]
answers = ["Hama dan penyakit tumbuhan merupakan jenis organisme pengganggu tumbuhan (OPT), selain gulma.",
           "Belalang, lalat buah, ulat grayak, penyakit antaknose, fuso, penyakit virus kerdil, busuk buah adalah beberapa hama dan penyakit tanaman yang paling merusak.",
           "Tiga cara penyebaran hama dan penyakit tanaman yaitu dengan cara: 1) perdagangan atau migrasi 2) pengaruh lingkungan, seperti faktor cuaca, angin, percikan air hujan, dan 3) faktor biotik berupa: serangga atau vektor lainnya."
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)




  #_________________________________________________________________________________________________#




context = "Permasalahan organisme pengganggu tumbuhan di lapangan seperti serangga dan penyakit dapat dicegah dengan menjaga kesehatan tanaman yaitu budidaya tanaman sehat, misalnya dengan menggunakan benih bersertifikat, memilih varietas tanaman yang tahan penyakit, melakukan rotasi tanaman yang bukan satu famili dengan tanaman yang ditanam di area yang sama selama beberapa tahun berturut-turut, mempertahankan nutrisi tanah seperti pemberian pupuk dengan dosis yang tepat. Pengolahan tanaman juga dapat dilakukan dengan cara irigasi yaitu mengairi tanaman dengan benar. Menjaga agar pertanaman bebas dari gulma, karena gulma dapat berpotensi menjadi sumber inokulum bagi patogen dan merupakan inang alternatif bagi hama dan penyakit yang menyerang pertanaman. Penggunaan pestisida kimia merupakan alternatif terakhir, karena penggunaan pestisida yang terlalu sering akan berdampak negatif bagi lingkungan dan dapat membunuh serangga yang menguntungkan. Hama merupakan salah satu jenis organisme pengganggu tanaman (OPT) yang karena aktivitasnya dapat merusak tanaman dan menyebabkan kerugian pada tanaman. Serangan hama ini dapat terjadi mulai dari benih hingga pasca panen (hama gudang). Pengertian hama dalam arti luas merupakan semua macam bentuk gangguan yang dapat menyebabkan kerusakan dan kerugian pada manusia, ternak dan tanaman. Sedangkan pengertian hama dalam arti sempit yaitu semua jenis hewan yang berpotensi mengganggu pada kegiatan budidaya tanaman yang berakibat merusak tanaman dan menurunkan produksi tanaman secara ekonomis (Simluhtan Kementerian Pertanian, 2019)."
queries = ["Apa yang dimaksud dengan hama dalam arti luas?",
           "Kapan serangan hama dapat terjadi pada tanaman?",
           "Apa yang dimaksud dengan hama dalam arti sempit?",
           "Apa dampak merusak tanaman yang disebabkan oleh hama?"
           ]
answers = ["Pengertian hama dalam arti luas merupakan semua macam bentuk gangguan yang dapat menyebabkan kerusakan dan kerugian pada manusia, ternak, dan tanaman.",
           "Serangan hama dapat terjadi mulai dari benih hingga pasca panen (hama gudang).",
           "Tiga cara penyebaran hama dan penyakit tanaman yaitu dengan cara: 1) perdagangan atau migrasi 2) pengaruh lingkungan, seperti faktor cuaca, angin, percikan air hujan, dan 3) faktor biotik berupa: serangga atau vektor lainnya."
           "Pengertian hama dalam arti sempit yaitu semua jenis hewan yang berpotensi mengganggu pada kegiatan budidaya tanaman yang berakibat merusak tanaman dan menurunkan produksi tanaman secara ekonomis.",
           "Aktivitas hama dapat merusak tanaman dan menyebabkan kerugian pada tanaman, menurunkan produksi tanaman secara ekonomis."
           ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: Apa yang dimaksud dengan Hama dan Penyakit Tanaman?
Prediction: merupakan
True Answer: Hama dan penyakit tumbuhan merupakan jenis organisme pengganggu tumbuhan (OPT), selain gulma.
EM: 0
F1: 0.15384615384615385
Precision: 1.0
Recall: 0.08333333333333333


Question: Apa saja beberapa hama dan penyakit tanaman yang merusak?
Prediction: 
True Answer: Belalang, lalat buah, ulat grayak, penyakit antaknose, fuso, penyakit virus kerdil, busuk buah adalah beberapa hama dan penyakit tanaman yang paling merusak.
EM: 0
F1: 0
Precision: 0
Recall: 0


Question: Bagaimana cara penyebaran hama dan penyakit tanaman?
Prediction: dengan cara : 1 ) perdagangan atau migrasi
True Answer: Tiga cara penyebaran hama dan penyakit tanaman yaitu dengan cara: 1) perdagangan atau migrasi 2) pengaruh lingkungan, seperti faktor cuaca, angin, percikan air hujan, dan 3) faktor biotik berupa: serangga atau vektor lainnya.
EM: 0
F1: 0.3076923076923077
Precision: 1.0
Recall: 0.18181818181818182


Question: Apa 